Forked from:https://www.kaggle.com/kenseitrg/simple-fastai-exercise

https://www.kaggle.com/interneuron/fast-fastai-with-condensenet

In [1]:
!pip install pytorchcv

    100% |████████████████████████████████| 286kB 15.3MB/s ta 0:00:01


In [2]:
!pip install fastai==1.0.47

    100% |████████████████████████████████| 215kB 28.1MB/s ta 0:00:01
  Found existing installation: fastai 1.0.50.post1
    Uninstalling fastai-1.0.50.post1:
      Successfully uninstalled fastai-1.0.50.post1


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pytorchcv.model_provider import get_model as ptcv_get_model

In [4]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch

In [5]:
data_folder = Path("../input")
#data_folder.ls()

In [6]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/sample_submission.csv")

In [7]:
test_img = ImageList.from_df(test_df, path=data_folder/'test', folder='test')
trfm = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)
train_img = (ImageList.from_df(train_df, path=data_folder/'train', folder='train')
        .split_by_rand_pct(0.01)
        .label_from_df()
        .add_test(test_img)
        .transform(trfm, size=128)
        .databunch(path='.', bs=64, device= torch.device('cuda:0'))
        .normalize(imagenet_stats)
       )

In [8]:
#train_img.show_batch(rows=3, figsize=(7,6))

In [9]:
def md(f=None):
    mdl = ptcv_get_model('condensenet74_c4_g4', pretrained=True)
    mdl.features.final_pool = nn.AvgPool2d(kernel_size=7, stride=1, padding=3)
    return mdl

In [10]:
# learn = cnn_learner(train_img, models.densenet161, metrics=[error_rate, accuracy])
learn = cnn_learner(train_img, md, metrics=[error_rate, accuracy])

In [11]:
#learn.lr_find()
#learn.recorder.plot()

In [12]:
lr = 3.5e-02
learn.fit_one_cycle(5, slice(lr))

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.048956,0.131776,0.034286,0.965714,01:07
1,0.029645,0.000703,0.000000,1.000000,01:02
2,0.028624,0.063166,0.011429,0.988571,01:02
3,0.009025,0.048415,0.011429,0.988571,01:01
4,0.006358,0.023773,0.005714,0.994286,01:03


In [13]:
#learn.unfreeze()
#learn.lr_find()
#learn.recorder.plot()

In [14]:
#learn.fit_one_cycle(1, slice(1e-06))

In [15]:
# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_top_losses(9, figsize=(7,6))

In [16]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test)

In [17]:
test_df.has_cactus = preds.numpy()[:, 0]

In [18]:
test_df.to_csv('simple_fastai_submission.csv', index=False)